In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
# import lightgbm
from lightgbm import LGBMRegressor
import datetime

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
rcParams['figure.figsize'] = (16, 8)
rc('font', family='AppleGothic')

In [13]:
train_path = './trainingdata'
infer_path = './inferencedata'
submission_path = './submission'
pil_path = './magok'

In [14]:
submission = pd.read_csv('./data/sample_submission.csv')
submission2 = pd.read_csv('./data/sample_submission.csv')

In [15]:
time_str = '20210105_091239'
train = pd.read_csv(
    os.path.join(train_path, f'training_{time_str}.csv')
)
X_test = pd.read_csv(
    os.path.join(infer_path, f'inference_{time_str}.csv')
)

In [23]:
train_day23 = pd.read_csv(
    os.path.join(train_path, f'training_{time_str}_day23.csv')
)
X_test_day23 = pd.read_csv(
    os.path.join(infer_path, f'inference_{time_str}_day23.csv')
)

In [24]:
train['Hour'] = train['Hour'].astype('category')
X_test['Hour'] = X_test['Hour'].astype('category')

In [25]:
X_train = train.drop(columns = ['1day_after_target', '2day_after_target'])
target_1 = train['1day_after_target']
target_2 = train['2day_after_target']

In [27]:
print(train.shape, X_test.shape)

(52272, 49) (3888, 47)


# Metric

In [28]:
def pinball(pred, actual):
    quantile_col = pred.columns.tolist()
    pred2 = pred.copy()
    actual2 = actual.copy()
    
    pred2.index = range(len(pred2))
    actual2.index = range(len(actual2))

    pinball_loss = []
    for item in quantile_col:
        pre_pinball = []
        for idx in range(len(pred2)):
            y = actual2.iloc[idx]
            z = pred2[item].iloc[idx]
            q = item
            
            if y >= z:
                value = (y - z)*q
                pre_pinball.append(value)
            else:
                value = (z - y)*(1 - q)
                pre_pinball.append(value)
        loss_mean = np.mean(pre_pinball)
        pinball_loss.append(loss_mean)
    
    fin = np.mean(pinball_loss)
    print(f'Pinball Loss: {fin}')
    
    return pinball_loss         

# Inference

In [29]:
X_train = train.drop(columns = ['1day_after_target', '2day_after_target'])
target_1 = train['1day_after_target']
target_2 = train['2day_after_target']

In [30]:
X_train_day23 = train_day23.drop(columns = ['1day_after_target', '2day_after_target'])
target_1_day23 = train_day23['1day_after_target']
target_2_day23 = train_day23['2day_after_target']

In [31]:
def LGBM_reg(X_train,y_train, test):
    quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    # 랜덤모수 샘플 추론이 왜 안되지;;;
    # fold = StratifiedKFold(n_splits=5, shuffle = True)
    
    fold = KFold(n_splits=7, shuffle = True)

    oof_preds = np.zeros([X_train.shape[0], len(quantiles)])
    
    feature_importance_df = pd.DataFrame()
    
    test_df = pd.DataFrame(np.zeros([test.shape[0], len(quantiles)]))
    test_df.columns = quantiles
    
    fold_metric = []
    # training
    target = y_train
    for fold_, (train_idx, valid_idx) in enumerate(
        fold.split(np.array(X_train), target)
    ):

        train_x, train_y = X_train.iloc[train_idx], target.iloc[train_idx]
        valid_x, valid_y = X_train.iloc[valid_idx], target.iloc[valid_idx]

        oof_df = pd.DataFrame()
        for idx, q in enumerate(quantiles):
            print(f'\nquantile: {q}\n')
            model = LGBMRegressor(objective='quantile',
                                  alpha=q,
                                  n_estimators=10000, 
                                  bagging_fraction=0.7, 
                                  learning_rate=0.027, 
                                  subsample=0.7
                                 )   

            model.fit(train_x, train_y, eval_metric = ['quantile'], 
              eval_set=[(valid_x, valid_y)], early_stopping_rounds=300, verbose=1500)

            oof_preds[[valid_idx],idx] = \
                model.predict(valid_x).round(2)

            oof_df[q] = model.predict(valid_x).round(2)
            test_df[q] += model.predict(test).round(2) / fold.n_splits
            
        metric = pinball(oof_df, valid_y)
        loss = np.mean(metric)
        print(f'\n FOLD {fold_}의 total pinball loss: {loss}\n')
        fold_metric.append(metric)
        # feature importance
        
        print('\n===================================================')
        print(f'FOLD {fold_} Success')
        print('===================================================\n')
        
    return oof_preds, fold_metric, loss, test_df

In [34]:
oof_preds, fold_metric, loss1, test_df = \
    LGBM_reg(X_train, target_1, X_test)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.25068
Early stopping, best iteration is:
[2151]	valid_0's quantile: 1.2448

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.86979
[3000]	valid_0's quantile: 1.83484
Early stopping, best iteration is:
[3849]	valid_0's quantile: 1.82976

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.21492
[3000]	valid_0's quantile: 2.15131
[4500]	valid_0's quantile: 2.1228
[6000]	valid_0's quantile: 2.10415
[7500]	valid_0's quantile: 2.09435
[9000]	valid_0's quantile: 2.08592
Did not meet early stopping. Best iteration is:
[9973]	valid_0's quantile: 2.08233

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.29124
[3000]	valid_0's quantile: 2.23771
[4500]	valid_0's quantile: 2.20742
[6000]	valid_0's quantile: 2.18658
[7500]	valid

[1500]	valid_0's quantile: 2.01901
[3000]	valid_0's quantile: 1.98386
[4500]	valid_0's quantile: 1.96856
Early stopping, best iteration is:
[5651]	valid_0's quantile: 1.96308

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.3532
[3000]	valid_0's quantile: 2.28887
[4500]	valid_0's quantile: 2.26128
[6000]	valid_0's quantile: 2.24016
[7500]	valid_0's quantile: 2.22823
[9000]	valid_0's quantile: 2.2147
Did not meet early stopping. Best iteration is:
[9995]	valid_0's quantile: 2.21007

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.44957
[3000]	valid_0's quantile: 2.38433
[4500]	valid_0's quantile: 2.34602
[6000]	valid_0's quantile: 2.3152
[7500]	valid_0's quantile: 2.28994
[9000]	valid_0's quantile: 2.27868
Did not meet early stopping. Best iteration is:
[9983]	valid_0's quantile: 2.26895

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	v

[3000]	valid_0's quantile: 1.89861
[4500]	valid_0's quantile: 1.88315
[6000]	valid_0's quantile: 1.87368
Early stopping, best iteration is:
[7105]	valid_0's quantile: 1.8675

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.23812
[3000]	valid_0's quantile: 2.19137
[4500]	valid_0's quantile: 2.16321
[6000]	valid_0's quantile: 2.1415
[7500]	valid_0's quantile: 2.13211
[9000]	valid_0's quantile: 2.12257
Did not meet early stopping. Best iteration is:
[9946]	valid_0's quantile: 2.11942

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.36026
[3000]	valid_0's quantile: 2.27959
[4500]	valid_0's quantile: 2.23928
[6000]	valid_0's quantile: 2.2231
[7500]	valid_0's quantile: 2.20658
[9000]	valid_0's quantile: 2.19509
Did not meet early stopping. Best iteration is:
[9961]	valid_0's quantile: 2.1916

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	va

In [35]:
oof_preds2, fold_metric2, loss2, test_df2 = \
    LGBM_reg(X_train, target_2, X_test)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.35039
Early stopping, best iteration is:
[2082]	valid_0's quantile: 1.34825

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.06487
Early stopping, best iteration is:
[2659]	valid_0's quantile: 2.03334

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.40964
[3000]	valid_0's quantile: 2.33246
[4500]	valid_0's quantile: 2.30179
[6000]	valid_0's quantile: 2.28897
[7500]	valid_0's quantile: 2.2811
[9000]	valid_0's quantile: 2.27486
Did not meet early stopping. Best iteration is:
[9762]	valid_0's quantile: 2.27041

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.50386
[3000]	valid_0's quantile: 2.4405
[4500]	valid_0's quantile: 2.40815
[6000]	valid_0's quantile: 2.39877
[7500]	valid_0's quantile: 2.38362
[9000]	valid

[1500]	valid_0's quantile: 1.33235
Early stopping, best iteration is:
[2313]	valid_0's quantile: 1.31998

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.0115
[3000]	valid_0's quantile: 1.97242
[4500]	valid_0's quantile: 1.95637
[6000]	valid_0's quantile: 1.9406
[7500]	valid_0's quantile: 1.93487
Early stopping, best iteration is:
[8493]	valid_0's quantile: 1.93193

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.33568
[3000]	valid_0's quantile: 2.25677
[4500]	valid_0's quantile: 2.22147
[6000]	valid_0's quantile: 2.20394
[7500]	valid_0's quantile: 2.18866
[9000]	valid_0's quantile: 2.17631
Did not meet early stopping. Best iteration is:
[10000]	valid_0's quantile: 2.16988

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.43652
[3000]	valid_0's quantile: 2.35218
[4500]	valid_0's quantile: 2.31621
[6000]	valid_0's qu

[1500]	valid_0's quantile: 1.31616
Early stopping, best iteration is:
[1784]	valid_0's quantile: 1.31425

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.01732
[3000]	valid_0's quantile: 1.98248
Early stopping, best iteration is:
[4144]	valid_0's quantile: 1.97098

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.36237
[3000]	valid_0's quantile: 2.30044
[4500]	valid_0's quantile: 2.2735
[6000]	valid_0's quantile: 2.25517
Early stopping, best iteration is:
[6666]	valid_0's quantile: 2.24739

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.49824
[3000]	valid_0's quantile: 2.41187
[4500]	valid_0's quantile: 2.37041
[6000]	valid_0's quantile: 2.34603
[7500]	valid_0's quantile: 2.33189
[9000]	valid_0's quantile: 2.32552
Did not meet early stopping. Best iteration is:
[9998]	valid_0's quantile: 2.32182

quantile: 0.5

Tra

In [36]:
oof_preds_day23, fold_metric_day23, loss3, test_df_day23 = \
    LGBM_reg(X_train_day23, target_1_day23, X_test_day23)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.35876
Early stopping, best iteration is:
[2019]	valid_0's quantile: 1.35341

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.06275
[3000]	valid_0's quantile: 2.0029
[4500]	valid_0's quantile: 1.9767
Early stopping, best iteration is:
[5349]	valid_0's quantile: 1.96625

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.36227
[3000]	valid_0's quantile: 2.29734
[4500]	valid_0's quantile: 2.26803
[6000]	valid_0's quantile: 2.25421
[7500]	valid_0's quantile: 2.24058
[9000]	valid_0's quantile: 2.225
Did not meet early stopping. Best iteration is:
[10000]	valid_0's quantile: 2.21768

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.43731
[3000]	valid_0's quantile: 2.35216
[4500]	valid_0's quantile: 2.31102
[6000]	valid_

[1500]	valid_0's quantile: 1.31718
Early stopping, best iteration is:
[2401]	valid_0's quantile: 1.30804

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.98999
[3000]	valid_0's quantile: 1.95618
[4500]	valid_0's quantile: 1.93598
[6000]	valid_0's quantile: 1.91839
Early stopping, best iteration is:
[6051]	valid_0's quantile: 1.91789

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.33783
[3000]	valid_0's quantile: 2.27041
[4500]	valid_0's quantile: 2.23796
[6000]	valid_0's quantile: 2.21727
[7500]	valid_0's quantile: 2.20112
[9000]	valid_0's quantile: 2.1852
Did not meet early stopping. Best iteration is:
[9992]	valid_0's quantile: 2.17884

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.44359
[3000]	valid_0's quantile: 2.37008
[4500]	valid_0's quantile: 2.32259
[6000]	valid_0's quantile: 2.29748
[7500]	valid_0's qu

Early stopping, best iteration is:
[597]	valid_0's quantile: 1.34348

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.02464
[3000]	valid_0's quantile: 1.98769
[4500]	valid_0's quantile: 1.96338
[6000]	valid_0's quantile: 1.95604
[7500]	valid_0's quantile: 1.95029
Early stopping, best iteration is:
[8023]	valid_0's quantile: 1.94861

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.36684
[3000]	valid_0's quantile: 2.29978
[4500]	valid_0's quantile: 2.27535
[6000]	valid_0's quantile: 2.25065
[7500]	valid_0's quantile: 2.23825
[9000]	valid_0's quantile: 2.23297
Did not meet early stopping. Best iteration is:
[9990]	valid_0's quantile: 2.22626

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.50181
[3000]	valid_0's quantile: 2.43893
[4500]	valid_0's quantile: 2.38856
[6000]	valid_0's quantile: 2.36149
[7500]	valid_0's qu

In [37]:
oof_preds2_day23, fold_metric2_day23,loss4, test_df2_day23 = \
    LGBM_reg(X_train_day23, target_2_day23, X_test_day23)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.35624
[3000]	valid_0's quantile: 1.34623
Early stopping, best iteration is:
[3010]	valid_0's quantile: 1.34616

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.0779
[3000]	valid_0's quantile: 2.02256
[4500]	valid_0's quantile: 2.00626
Early stopping, best iteration is:
[5194]	valid_0's quantile: 1.99833

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.38902
[3000]	valid_0's quantile: 2.30868
[4500]	valid_0's quantile: 2.27772
[6000]	valid_0's quantile: 2.25483
[7500]	valid_0's quantile: 2.24294
[9000]	valid_0's quantile: 2.23138
Did not meet early stopping. Best iteration is:
[9985]	valid_0's quantile: 2.22332

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.50718
[3000]	valid_0's quantile: 2.41431
[4500]	vali

[3000]	valid_0's quantile: 0.75148
Early stopping, best iteration is:
[4007]	valid_0's quantile: 0.751313
Pinball Loss: 1.7899251590100151

 FOLD 2의 total pinball loss: 1.7899251590100151


FOLD 2 Success


quantile: 0.1

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.3141
Early stopping, best iteration is:
[1522]	valid_0's quantile: 1.31359

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.03592
[3000]	valid_0's quantile: 1.9617
[4500]	valid_0's quantile: 1.93293
[6000]	valid_0's quantile: 1.9204
Early stopping, best iteration is:
[6818]	valid_0's quantile: 1.91058

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.29781
[3000]	valid_0's quantile: 2.2315
[4500]	valid_0's quantile: 2.21015
[6000]	valid_0's quantile: 2.18034
[7500]	valid_0's quantile: 2.1633
[9000]	valid_0's quantile: 2.15449
Did not meet early stopping. Best iterat


quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.32347
[3000]	valid_0's quantile: 1.30768
[4500]	valid_0's quantile: 1.30256
[6000]	valid_0's quantile: 1.30211
Early stopping, best iteration is:
[5700]	valid_0's quantile: 1.30076

quantile: 0.9

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 0.742254
Early stopping, best iteration is:
[2217]	valid_0's quantile: 0.74147
Pinball Loss: 1.7403111109689453

 FOLD 5의 total pinball loss: 1.7403111109689453


FOLD 5 Success


quantile: 0.1

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.29701
[3000]	valid_0's quantile: 1.28301
Early stopping, best iteration is:
[3258]	valid_0's quantile: 1.28152

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.96966
[3000]	valid_0's quantile: 1.92707
Early stopping, best iteration is:
[3234]	valid_0's quantile: 

# validation score

In [38]:
day7 = np.mean(loss1)
day8 = np.mean(loss2)
final = np.mean([day7, day8])
print(f'day7 pinball loss: {day7}')
print(f'day8 pinball loss: {day8}')
print(f'Total pinball loss: {final}')

day7 pinball loss: 1.7049203213296877
day8 pinball loss: 1.8067876808695145
Total pinball loss: 1.755854001099601


In [39]:
day7 = np.mean(loss3)
day8 = np.mean(loss4)
final = np.mean([day7, day8])
print(f'day7 pinball loss: {day7}')
print(f'day8 pinball loss: {day8}')
print(f'Total pinball loss: {final}')

day7 pinball loss: 1.7911621863953604
day8 pinball loss: 1.7327504755004093
Total pinball loss: 1.7619563309478847


# submission

In [40]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = test_df.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = test_df2.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
submission2.loc[submission2.id.str.contains("Day7"), "q_0.1":] = test_df_day23.sort_index().values
submission2.loc[submission2.id.str.contains("Day8"), "q_0.1":] = test_df2_day23.sort_index().values
submission2

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 제출

In [42]:
# time_str = datetime.datetime.strftime(
#     datetime.datetime.today(),
#     '%Y%m%d_%H%M%S'
# )

# training 데이터 저장한 시간이랑 맞춰서 사용
submission.to_csv(
    os.path.join(submission_path, f'제출용_{time_str}.csv'), 
    index=False
)

# Ensemble

In [43]:
submission3 = submission[['id']]
sub_value = submission.iloc[:,1:] * 0.7 + submission2.iloc[:,1:] * 0.3

submission3 = pd.concat([submission3,sub_value],axis = 1)

In [44]:
# 앙상블 결과 제출
submission3.to_csv(
    os.path.join(submission_path, f'제출용_{time_str}_ensemble_가중평균.csv'), 
    index=False
)

In [47]:
import pandas as pd
submission3 = pd.read_csv(submission_path + "/20210105_091239.csv")

In [36]:
print(submission3.columns.tolist())

['id', 'q_0.1', 'q_0.2', 'q_0.3', 'q_0.4', 'q_0.5', 'q_0.6', 'q_0.7', 'q_0.8', 'q_0.9']


In [ ]:
# column의 값을 정렬하자 

In [50]:
from tqdm.notebook import tqdm
for i in tqdm(range(0, len(submission3))):
    submission3.loc[i, columns] = sorted(submission3.loc[i, columns].values)

In [51]:
columns = [ 'q_0.1', 'q_0.2', 'q_0.3', 'q_0.4', 'q_0.5', 'q_0.6', 'q_0.7', 'q_0.8', 'q_0.9']
for i in columns: 
    submission3.loc[submission3[i] < 0, i] = 0

In [52]:
# 앙상블 결과 제출
submission3.to_csv(
    os.path.join(submission_path, f'{time_str}3.csv'), 
    index=False
)

In [54]:
(2.5).round(2)

AttributeError: 'float' object has no attribute 'round'